# basic dynamodb with boto3

* configure AWS Settings

In [1]:
from ipywidgets import widgets
from IPython.display import display
access_key = widgets.Text(description="access key",width=400)
secret_access_key = widgets.Text(description="secret key",width=400)
display(access_key)
display(secret_access_key)

* print AWS Settings

In [ ]:
#print access_key.value
#print secret_access_key.value

* create AWS session

In [2]:
import boto3
from boto3.session import Session

session = Session(aws_access_key_id=access_key.value,
                  aws_secret_access_key=secret_access_key.value,
                  region_name='ap-northeast-1')

* get dynamodb resource

In [3]:
dynamodb = session.resource('dynamodb')

* display dynamodb tables and drop table

In [4]:
tables = dynamodb.tables.all()
for table in tables:
    print table.name
    if table.name == 'coffee':
        table.delete()

coffee


* create new coffee table

In [5]:
table = dynamodb.create_table(
    AttributeDefinitions=[
        {
            'AttributeName': 'user_id',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'time',
            'AttributeType': 'S'
        },
    ],
    TableName='coffee',
    KeySchema=[
        {
            'AttributeName': 'user_id',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'time',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 30,
        'WriteCapacityUnits': 30
    }
)

* insert data into coffee table

In [6]:
import datetime

response = table.put_item(
   Item={
        'user_id': 1,
        'time': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'info': 'test1'
    }
)
response = table.put_item(
   Item={
        'user_id': 2,
        'time': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'info': 'test2'
    }
)
response = table.put_item(
   Item={
        'user_id': 3,
        'time': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'info': 'test3'
    }
)

* query

In [7]:
from boto3.dynamodb.conditions import Key, Attr

response = table.query(
    KeyConditionExpression=Key('user_id').eq(1)
)

for i in response['Items']:
    print(i['user_id'], ":", i['time'], ":", i['info'])

(Decimal('1'), ':', u'2017-04-17 10:05:25', ':', u'test1')


* get all items

In [8]:
response = table.scan()
for i in response['Items']:
    print(i['user_id'], ":", i['time'], ":", i['info'])

(Decimal('3'), ':', u'2017-04-17 10:05:25', ':', u'test3')
(Decimal('2'), ':', u'2017-04-17 10:05:25', ':', u'test2')
(Decimal('1'), ':', u'2017-04-17 10:05:25', ':', u'test1')


* get item

In [9]:
response = table.get_item(
    Key={
        'user_id': 1,
        'time': '2017-04-17 10:05:25'
    }
)
print(response['Item']['user_id'], ":", response['Item']['time'], ":", response['Item']['info'])

(Decimal('1'), ':', u'2017-04-17 10:05:25', ':', u'test1')


* create json decoder function

In [10]:
import json
import decimal
#Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

* convert data to json

In [11]:
item = response['Item']
print(json.dumps(item, indent=4, cls=DecimalEncoder))

{
    "info": "test1", 
    "user_id": 1, 
    "time": "2017-04-17 10:05:25"
}


* delete item

In [12]:
response = table.delete_item(
    Key={
        'user_id': 1,
        'time': '2017-04-17 10:05:25'
    }
) 